In [1]:
import os
import time
import math
import helper
import requests
import bs4 as bs
import numpy as np
import pandas as pd
import pickle as pkl
import datetime as dt
import tensorflow as tf
from sklearn import metrics
from matplotlib import style
import yahoo_finance as yahoo
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.finance import candlestick_ohlc
from IPython.display import display, Math, Latex
style.use('ggplot')

C:\Users\Khanh\Anaconda3\envs\py35\lib\site-packages\matplotlib\cbook.py:136: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [2]:
# def get_index_yahoo(ticker_name):
#     getter = yahoo.Share(ticker_name)
#     index = getter.get_historical('2000-1-1', '2016-12-31')
#     date = np.array([index[i]['Date'] for i in range(len(index))])
#     date = np.flip(date, axis=0)
#     close = np.array([index[i]['Close'] for i in range(len(index))])
#     close = np.flip(close, axis=0)
#     close = close.astype(float)
#     date = pd.to_datetime(date)
#     return close, date
# djia_close, djia_date = get_index_yahoo('^DJI')
# ndx_close, ndx_date = get_index_yahoo('^NDX')
# # sp100_close, sp100_date = get_index_yahoo('^SP100') # cannot download
# sp500_close, sp500_date = get_index_yahoo('^GSPC')
# # dow_retail_close, dow_retail_date = get_index_yahoo('^DJUSRT') # cannot download

In [4]:
# ibm = pd.read_csv('./ibm/ibm.csv')
# ibm['Date'] = pd.to_datetime(ibm['Date'])
# ibm.set_index('Date', inplace=True)
# ibm = ibm.loc['2000-01-01':'2016-12-31']
# ibm['Date']= pd.Series(djia_date, index=ibm.index)
# ibm['NDX'] = pd.Series(ndx_close, index=ibm.index)
# ibm['DJIA'] = pd.Series(djia_close, index=ibm.index)
# ibm['SP500'] = pd.Series(sp500_close, index=ibm.index)
# ibm.head()

,Open,High,Low,Close,Volume,Adj Close,Date,NDX,DJIA,SP500
Date,,,,,,,,,,
2000-03-01,112.4375,116.0000,111.875,116.0000,10347700,87.761136,2000-01-03,3790.550049,11357.509766,1455.219971
2000-04-01,114.0000,114.5000,110.875,112.0625,8227800,84.782175,2000-01-04,3546.199951,10997.929688,1399.420044
2000-05-01,112.9375,119.7500,112.125,116.0000,12733200,87.761136,2000-01-05,3507.310059,11122.650391,1402.109985
2000-06-01,118.0000,118.9375,113.500,114.0000,7971900,86.248013,2000-01-06,3340.810059,11253.259766,1403.449951
2000-07-01,117.2500,117.9375,110.625,113.5000,11856700,85.869732,2000-01-07,3529.600098,11522.559570,1441.469971


In [4]:
year = {
    2000: ibm.loc['2000-01-01':'2000-12-31'].index,
    2001: ibm.loc['2001-01-01':'2001-12-31'].index,
    2002: ibm.loc['2002-01-01':'2002-12-31'].index,
    2003: ibm.loc['2003-01-01':'2003-12-31'].index,
    2004: ibm.loc['2004-01-01':'2004-12-31'].index,
    2005: ibm.loc['2005-01-01':'2005-12-31'].index,
    2006: ibm.loc['2006-01-01':'2006-12-31'].index,
    2007: ibm.loc['2007-01-01':'2007-12-31'].index,
    2008: ibm.loc['2008-01-01':'2008-12-31'].index,
    2009: ibm.loc['2009-01-01':'2009-12-31'].index,
    2010: ibm.loc['2010-01-01':'2010-12-31'].index,
    2011: ibm.loc['2011-01-01':'2011-12-31'].index,
    2012: ibm.loc['2012-01-01':'2012-12-31'].index,
    2013: ibm.loc['2013-01-01':'2013-12-31'].index,
    2014: ibm.loc['2014-01-01':'2014-12-31'].index,
    2015: ibm.loc['2015-01-01':'2015-12-31'].index,
    2016: ibm.loc['2016-01-01':'2016-12-31'].index,
}

In [7]:
# ibm.to_csv('./ibm/full_ibm.csv')

In [2]:
# df = pd.read_csv('./ibm/full_ibm.csv')
# df['Date.1'] = pd.to_datetime(df['Date.1'])
# df.set_index('Date.1', inplace=True)
# df.drop(['Date'], axis=1, inplace=True)
# df.head()

,Open,High,Low,Close,Volume,Adj Close,NDX,DJIA,SP500
Date.1,,,,,,,,,
2000-01-03,112.4375,116.0000,111.875,116.0000,10347700,87.761136,3790.550049,11357.509766,1455.219971
2000-01-04,114.0000,114.5000,110.875,112.0625,8227800,84.782175,3546.199951,10997.929688,1399.420044
2000-01-05,112.9375,119.7500,112.125,116.0000,12733200,87.761136,3507.310059,11122.650391,1402.109985
2000-01-06,118.0000,118.9375,113.500,114.0000,7971900,86.248013,3340.810059,11253.259766,1403.449951
2000-01-07,117.2500,117.9375,110.625,113.5000,11856700,85.869732,3529.600098,11522.559570,1441.469971


In [3]:
# def calculate_technical_indicator(data):
#     data['ewma_5'] = data['Close'].ewm(span=5).mean()
#     data['ewma_22'] = data['Close'].ewm(span=22).mean()
#     data['ewma_200'] = data['Close'].ewm(span=200).mean()
#     data['MACD'] = data['Close'].ewm(span=12).mean() - data['Close'].ewm(span=26).mean()
#     data['Lower band'] = data['Close'].rolling(window=20).mean() - 2*data['Close'].rolling(window=20).std()
#     data['Upper band'] = data['Close'].rolling(window=20).mean() + 2*data['Close'].rolling(window=20).std()
#     data['Middle band'] = data['Close'].rolling(window=20).mean()
#     data['AO'] = ((data['High'] + data['Low'])/2).rolling(window=5).mean() - ((data['High'] + data['Low'])/2).rolling(window=34).mean()
#     data['AC'] =  data['AO'] - data['AO'].rolling(window=5).mean()
#     data['%R'] = (data['High'].max() - data['Close'])*100/(data['High'].max()-data['Low'].min())
#     return data
   
# df = calculate_technical_indicator(df)
# df.head()
# # ibm['ewma_5'] = ibm['Close'].ewm(span=5).mean()
# # ibm['ewma_22'] = ibm['Close'].ewm(span=22).mean()
# # ibm['ewma_200'] = ibm['Close'].ewm(span=200).mean()
# # ibm['MACD'] = ibm['Close'].ewm(span=12).mean() - ibm['Close'].ewm(span=26).mean()
# # ibm['Lower band'] = ibm['Close'].rolling(window=20).mean() - 2*ibm['Close'].rolling(window=20).std()
# # ibm['Upper band'] = ibm['Close'].rolling(window=20).mean() + 2*ibm['Close'].rolling(window=20).std()
# # ibm['Middle band'] = ibm['Close'].rolling(window=20).mean()
# # ibm['AO'] = ((ibm['High'] + ibm['Low'])/2).rolling(window=5).mean() - ((ibm['High'] + ibm['Low'])/2).rolling(window=34).mean()
# # ibm['AC'] =  ibm['AO'] - ibm['AO'].rolling(window=5).mean()
# # ibm['%R'] = (ibm['High'].max() - ibm['Close'])*100/(ibm['High'].max()-ibm['Low'].min())
# # ibm.head()

,Open,High,Low,Close,Volume,Adj Close,NDX,DJIA,SP500,ewma_5,ewma_22,ewma_200,MACD,Lower band,Upper band,Middle band,AO,AC,%R
Date.1,,,,,,,,,,,,,,,,,,,
2000-01-03,112.4375,116.0000,111.875,116.0000,10347700,87.761136,3790.550049,11357.509766,1455.219971,116.000000,116.000000,116.000000,0.000000,NaN,NaN,NaN,NaN,NaN,61.708565
2000-01-04,114.0000,114.5000,110.875,112.0625,8227800,84.782175,3546.199951,10997.929688,1399.420044,113.637500,113.941761,114.021406,-0.088341,NaN,NaN,NaN,NaN,NaN,64.140772
2000-01-05,112.9375,119.7500,112.125,116.0000,12733200,87.761136,3507.310059,11122.650391,1402.109985,114.756579,114.691113,114.687544,0.009537,NaN,NaN,NaN,NaN,NaN,61.708565
2000-01-06,118.0000,118.9375,113.500,114.0000,7971900,86.248013,3340.810059,11253.259766,1403.449951,114.442308,114.494094,114.513071,-0.018389,NaN,NaN,NaN,NaN,NaN,62.943972
2000-01-07,117.2500,117.9375,110.625,113.5000,11856700,85.869732,3529.600098,11522.559570,1441.469971,114.080569,114.257564,114.306384,-0.053072,NaN,NaN,NaN,NaN,NaN,63.252824


In [4]:
df.shape

(4277, 19)

In [5]:
# temporal_windows = 15
# def prepare_data_features(data, windows):
#     l = []
#     cols = data.columns
#     for w in range(windows):
#         shift = data.shift(periods=w+1, axis=0)
#         shift.columns = [(cols[i] + '_{}'.format(w+1)) for i in range(len(cols))]
#         l.append(shift)
#     output = pd.concat(l, axis=1)
#     output.dropna(axis=0, how='any', inplace=True)
#     return output
# horizons = 15
# def prepare_data_target(data, horizons):
#     cols = data.columns
#     l = []
#     for h in range(horizons):
#         shift = data.shift(periods=-h, axis=0)
#         shift.columns = [(cols[i] + '_{}'.format(h+1)) for i in range(len(cols))]
#         l.append(shift)
#     output = pd.concat(l, axis=1)
#     output.dropna(axis=0, how='any', inplace=True)
#     return output

# df_train = prepare_data_features(df, temporal_windows)

# df_target = prepare_data_target(df[['Close']].loc[df_train.index], horizons)

# df_train.tail()

,Open_1,High_1,Low_1,Close_1,Volume_1,Adj Close_1,NDX_1,DJIA_1,SP500_1,ewma_5_1,...,ewma_5_15,ewma_22_15,ewma_200_15,MACD_15,Lower band_15,Upper band_15,Middle band_15,AO_15,AC_15,%R_15
Date.1,,,,,,,,,,,,,,,,,,,,,
2016-12-23,167.360001,168.229996,166.580002,167.059998,2802600.0,165.749434,4934.390137,19918.880859,2260.959961,167.156125,...,161.211683,159.624787,153.888411,1.786577,153.358388,166.306613,159.832500,5.545266,-0.961112,34.517259
2016-12-27,167.000000,167.490005,166.449997,166.710007,1701200.0,165.402189,4940.020020,19933.810547,2263.790039,167.007419,...,160.754454,159.643501,153.947630,1.563274,154.744069,165.661932,160.203001,4.728706,-1.356201,34.628451
2016-12-28,166.979996,167.979996,166.850006,167.139999,1397500.0,165.828809,4965.810059,19945.039062,2268.879883,167.051612,...,160.619638,159.704936,154.011336,1.411191,155.399839,165.469163,160.434501,3.648910,-1.765991,34.313416
2016-12-29,167.289993,167.740005,166.000000,166.190002,1757500.0,164.886264,4926.290039,19833.679688,2249.919922,166.764409,...,162.009756,160.147115,154.118586,1.630142,156.168640,165.662361,160.915501,3.277115,-1.425044,31.570821
2016-12-30,166.020004,166.990005,166.000000,166.600006,1663500.0,165.293051,4918.279785,19819.779297,2249.260010,166.709608,...,163.126505,160.600410,154.230441,1.828579,157.239204,165.646797,161.443001,3.862350,-0.350119,31.218725


In [6]:
df_target.tail()

,Close_1,Close_2,Close_3,Close_4,Close_5,Close_6,Close_7,Close_8,Close_9,Close_10,Close_11,Close_12,Close_13,Close_14,Close_15
Date.1,,,,,,,,,,,,,,,
2016-12-05,159.839996,160.350006,164.789993,165.360001,166.520004,165.500000,168.289993,168.509995,168.020004,166.729996,166.679993,167.600006,167.330002,167.059998,166.710007
2016-12-06,160.350006,164.789993,165.360001,166.520004,165.500000,168.289993,168.509995,168.020004,166.729996,166.679993,167.600006,167.330002,167.059998,166.710007,167.139999
2016-12-07,164.789993,165.360001,166.520004,165.500000,168.289993,168.509995,168.020004,166.729996,166.679993,167.600006,167.330002,167.059998,166.710007,167.139999,166.190002
2016-12-08,165.360001,166.520004,165.500000,168.289993,168.509995,168.020004,166.729996,166.679993,167.600006,167.330002,167.059998,166.710007,167.139999,166.190002,166.600006
2016-12-09,166.520004,165.500000,168.289993,168.509995,168.020004,166.729996,166.679993,167.600006,167.330002,167.059998,166.710007,167.139999,166.190002,166.600006,165.990005


In [7]:
# def inter_polation_norm(x):
#     return (x-x.min())/(x.max() - x.min())
# def z_score_norm(x):
#     return (x - x.mean())/x.std()
# df_train = inter_polation_norm(df_train)
# df_target = inter_polation_norm(df_target)
# df_train = df_train.loc[df_target.index]
# print (df_train.shape)
# print (df_target.shape)

(4211, 285)
(4211, 15)


In [10]:
# ## Train test split
# x_train, y_train = df_train.loc[:'2014-01-01'].as_matrix(), df_target.loc[:'2014-01-01']
# x_test, y_test = df_train.loc['2014-01-01':].as_matrix(), df_target.loc['2014-01-01':]

In [11]:
# def reshape_target_df(data, horizons):
#     l = np.array(data.iloc[:,0])
#     for i in range(data.shape[0]):
#         l[i] = (np.mean([data.iloc[i-j, k] for j, k in zip(range(horizons), range(horizons)) if (i >= j)]))
#     return l
# def reshape_target_array(data, horizons):
#     l = np.array(data[:,0])
#     for i in range(data.shape[0]):
#         l[i] = (np.mean([data[i-j, k] for j, k in zip(range(horizons), range(horizons)) if (i >= j)]))
#     return l


In [2]:
df = pd.read_csv('./ibm/full_ibm.csv')
df['Date.1'] = pd.to_datetime(df['Date.1'])
df.set_index('Date.1', inplace=True)
df.drop(['Date','High', 'Adj Close', 'NDX', 'SP500', 'Volume', 'Low'], axis=1, inplace=True)
df['ma7'] = df['Close'].rolling(window=7).mean()
df['ma30'] = df['Close'].rolling(window=30).mean()
df['std30'] = df['Close'].rolling(window=30).std()
df_train = df.drop(['ma7', 'Close', 'ma30'],axis=1)
df_train['ma7_1'] = df['ma7'].shift(periods = 11, axis=0)
df_train['ma7_2'] = df['ma7'].shift(periods = 18, axis=0)
df_train['ma7_3'] = df['ma7'].shift(periods = 25, axis=0)
df_train['ma7_4'] = df['ma7'].shift(periods = 32, axis=0)
df_train['ma30_1'] = df['ma7'].shift(periods = 62, axis=0)
df_train['ma30_2'] = df['ma7'].shift(periods = 92, axis=0)
df_train['ma30_3'] = df['ma7'].shift(periods = 112, axis=0)
df_train['ma30_4'] = df['ma7'].shift(periods = 142, axis=0)
df_train['Close_1'] = df['Close'].shift(periods = 1, axis=0)
df_train['Close_2'] = df['Close'].shift(periods = 2, axis=0)
df_train['Close_3'] = df['Close'].shift(periods = 3, axis=0)


## Network

In [4]:
## params
N_FEATURES = 126 # number of input features
X = tf.placeholder(tf.float32, [None, N_FEATURES])
Y = tf.placeholder(tf.float32, [None, 1])

In [6]:
def new_fc_layer(in_layer, n_input_features, n_output_layer, activation=''):
    weights = tf.Variable(tf.truncated_normal([n_input_features, n_output_layer], stddev=0.1))
    biases = tf.Variable(tf.constant(value=1, dtype=tf.float32, shape = [n_output_layer]))
    out_layer = tf.matmul(in_layer, weights) + biases
    activation = activation.lower()
    if(activation == 'relu'):
        out_layer = tf.nn.relu(out_layer)
    elif(activation == 'tanh'):
        out_layer = tf.nn.tanh(out_layer)
    elif(activation == 'sigmoid'):
        out_layer = tf.nn.sigmoid(out_layer)
    return out_layer, weights, biases
## Hidden layer 1
N_HIDDEN_L1 = 50
hidden_layer_1, weight_1, biases_1 = new_fc_layer(X, N_FEATURES, N_HIDDEN_L1, 'tanh')
## Hidden layer 2
N_HIDDEN_L2 = 1
hidden_layer_2, weight_2, biases_2 = new_fc_layer(hidden_layer_1, N_HIDDEN_L1, N_HIDDEN_L2, 'tanh')

print (hidden_layer_1)
print (hidden_layer_2)

Tensor("Tanh_1:0", shape=(?, 50), dtype=float32)
Tensor("Tanh_2:0", shape=(?, 1), dtype=float32)


In [7]:
# global_step = tf.Variable(0, trainable=False)
# starter_learning_rate = 1e-3
# cost = tf.reduce_mean(tf.square(hidden_layer_2-Y))
# learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
#                                            50000, 0.9, staircase=True)
# optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost, global_step=global_step)
# y_pred = hidden_layer_2
# y_true = Y

In [8]:
# ## Cost function and optimization
# # BETA = 0.001 #L2 regularization penalty factor
# LEARNING_RATE = 1e-3
# cost = tf.reduce_mean(tf.square(hidden_layer_2-Y)) # To be added L2 regularization
# optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(cost)
# ## Making Prediction
# y_pred = hidden_layer_2
# y_true = Y

In [26]:
beta = 0.001
learning_rate = tf.placeholder(tf.float32, shape=[])
def decay_learning_rate(learning_step, total_step):
    if(learning_step <= 500):
        return 1e-2
    return 1e-2*(1-(learning_step-500)/total_step)

cost = tf.reduce_mean(tf.square(hidden_layer_1-Y) + beta*tf.nn.l2_loss(weight_1) )
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
y_pred = hidden_layer_2
y_true = Y

In [27]:
BATCH_SIZE = 50
TRAINING_EPOCHS = 20000

## Helper function for optimization
def optimize(train_x, train_y, test_x, test_y, n_epochs, batch_size, session, saver, dir_path):
    n_samples = train_x.shape[0]
    n_iterations = np.int(np.floor(n_samples/batch_size))+1
    start_time = time.time()
    cost_history = np.empty(shape=[1],dtype=float)
    metrics = np.zeros(shape=(np.int(TRAINING_EPOCHS // 1000)+2,3), dtype=float)
    train_y_matrix = train_y.as_matrix()
    print ("Training.......")
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    for epoch in np.arange(n_epochs+1):
        for itr in np.arange(n_iterations):
            start = (itr * batch_size) % (n_samples )
            batch_x, batch_y = train_x[start:start + batch_size], train_y_matrix[start:start + batch_size]
            lr = decay_learning_rate(epoch, n_epochs)
            feed_dict_train = {X: batch_x, Y: batch_y, learning_rate: lr}
            _, c = session.run([optimizer, cost], feed_dict=feed_dict_train)
        if(epoch >= 1000):
            cost_history = np.append(cost_history,c)
            
        if(epoch % 1000 == 0):
            print ("-- Elapsed time -- Epoch -- Cost value -- ")
            print ("-- {:12.6f} -- {:5d} -- {:10.5f} -- ".format((time.time() - start_time), 
                                                                                    epoch, 
                                                                                    c, 
                                                                                    ))
            print ("-- Making prediction at {}th epoch".format(epoch))
            r, m , h = make_prediction(test_x, test_y, sess, BATCH_SIZE, dir_path, epoch=epoch)
            counter = np.int(epoch // 1000)
            metrics[counter, 0] = r
            metrics[counter, 1] = m
            metrics[counter, 2] = h
            
        
#             Draw weights of convolutional layer
#             if(epoch % (n_epochs/2) == 0):
#                 plot_conv_weights(session, conv_weights[0], 'conv_1', 1, epoch)
#                 plot_conv_weights(session, conv_weights[1], 'conv_2', 1, epoch)
#                 plot_conv_weights(session, conv_weights[2], 'conv_3', 1, epoch)
                  
#         Save model in folder rnn_model
        
#         saver.save(sess, 'rnn_model/new_cnn')
        
#         print running time and output cost value graph
    print ("Making prediction on training set")
    r, m , h = make_prediction(train_x, train_y, sess, BATCH_SIZE, dir_path, 0, train=True)
    metrics[-1, 0] = r
    metrics[-1, 1] = m
    metrics[-1, 2] = h
    df_metrics = pd.DataFrame(data=metrics, index=np.arange(0,21001,1000), columns=["RMSE", "MAE", "Hit rate"])
    df_metrics.to_csv('{0}metrics.csv'.format(dir_path))
    print ("---Total Running time: %s seconds ---" % (time.time() - start_time))
    print ('*'*50)
    plt.close()
    plt.clf()
    fig = plt.figure(figsize=(10,5))
    cost_history = (cost_history - cost_history.min())/(cost_history.max() - cost_history.min())
    plt.plot(cost_history)
    plt.savefig('{0}Cost'.format(dir_path), dpi=1000)
    
## Helper function to print confusion matrix
def make_prediction(test_x, test_y, session, batch_size, dir_path, epoch, train=False):
    print ("Making prediction.......")
    start_time = time.time()
    n_samples = test_x.shape[0]
    n_iterations = np.int(np.floor(n_samples/batch_size))+1 
    pred = np.zeros((n_samples, test_y.shape[1]))
    true = test_y.as_matrix()
    for itr in np.arange(n_iterations):
        start = (itr * batch_size) % (n_samples)
        batch_x = test_x[start:start + batch_size]
        feed_dict_test = {X: batch_x}
        pred[start:start + batch_size] = session.run(y_pred, feed_dict=feed_dict_test)
    print (np.prod(np.isnan(pred)))
#     pred = reshape_target_array(pred, horizons)
#     true = reshape_target_array(true, horizons)
#     pred = (pred + 1) * (true.max() - true.min()) / 2 + true.min()
    rmse = math.sqrt(metrics.mean_squared_error(pred, true))
    mae = metrics.mean_absolute_error(pred, true)
    hit = np.mean(helper.step((pred[1:] - true[:-1])*(true[1:] - true[:-1])))
    print ("RMSE = {}".format(rmse))
    print ("MAE = {}".format(mae))
    print ("Hit rate = {}".format(hit))
    if(train):
        plt.close()
        plt.clf()
        fig, ax = plt.subplots(figsize=(10,5))
        ax.plot(test_y.index, pred, color = 'r', label='Predicted Values')
        ax.plot(test_y.index, true, color = 'b', label='Acutal Values')
        ax.legend(loc='upper right', shadow=False)
        plt.savefig('{0}Trend_training'.format(dir_path), dpi=1000)
    else:
        plt.close()
        plt.clf()
        fig, ax = plt.subplots(figsize=(10,5))
        ax.plot(test_y.index, pred, color = 'r', label='Predicted Values')
        ax.plot(test_y.index, true, color = 'b', label='Acutal Values')
        ax.legend(loc='upper right', shadow=False)
        plt.savefig('{0}Trend_{1}th_epoch'.format(dir_path, epoch), dpi=1000)
        plt.close()
        plt.clf()
        fig, ax = plt.subplots(figsize=(10,5))
        ax.plot(test_y.index, np.absolute(pred-true), color = 'r', label='Absolute error')
        plt.savefig('{0}Abs_error_{1}th_epoch'.format(dir_path, epoch), dpi=1000)
#     plt.show()
    print ("---Running Time: {0} seconds ---".format((time.time() - start_time)))
    print ('*'*50)
    return rmse, mae, hit



In [28]:
df_train_d = helper.inter_polation_norm_1_1(df_train)
df_train_data = helper.trignometric_expansion(df_train_d, 4)
df_target = df.loc[df_train_data.index][['Close']]
df_target = helper.inter_polation_norm_1_1(df_target)
## Train test split
x_train, y_train = df_train_data.loc[:'2014-01-01'].as_matrix(), df_target.loc[:'2014-01-01']
x_test  = df_train_data.loc['2014-01-01':].as_matrix()
d = './MLP_decay_tri_L2_1/'
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    optimize(x_train, y_train, x_test, df_target.loc['2014-01-01':] , TRAINING_EPOCHS, BATCH_SIZE, sess, saver, d)
    
    

Training.......
-- Elapsed time -- Epoch -- Cost value -- 
--     0.050537 --     0 --    0.25147 -- 
-- Making prediction at 0th epoch
Making prediction.......
0
RMSE = 0.5851087572568003
MAE = 0.5464597005035353
Hit rate = 0.4966887417218543
---Running Time: 4.722896575927734 seconds ---
**************************************************
-- Elapsed time -- Epoch -- Cost value -- 
--    44.759438 --  1000 --    0.18174 -- 
-- Making prediction at 1000th epoch
Making prediction.......
0
RMSE = 0.5804310340339022
MAE = 0.5414059697135888
Hit rate = 0.4966887417218543
---Running Time: 5.2906153202056885 seconds ---
**************************************************
-- Elapsed time -- Epoch -- Cost value -- 
--    89.573759 --  2000 --    0.18067 -- 
-- Making prediction at 2000th epoch
Making prediction.......
0
RMSE = 0.5293041176370453
MAE = 0.48649514166748875
Hit rate = 0.495364238410596
---Running Time: 4.793056964874268 seconds ---
**************************************************

In [29]:
df_train_data = helper.chebyshev_expansion(df_train_d, 8)
df_target = df.loc[df_train_data.index][['Close']]
df_target = helper.inter_polation_norm_1_1(df_target)
## Train test split
x_train, y_train = df_train_data.loc[:'2014-01-01'].as_matrix(), df_target.loc[:'2014-01-01']
x_test  = df_train_data.loc['2014-01-01':].as_matrix()
d = './MLP_decay_che_L2_1/'
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    optimize(x_train, y_train, x_test, df_target.loc['2014-01-01':] , TRAINING_EPOCHS, BATCH_SIZE, sess, saver, d)
    
    

Training.......
-- Elapsed time -- Epoch -- Cost value -- 
--     0.047152 --     0 --    0.22742 -- 
-- Making prediction at 0th epoch
Making prediction.......
0
RMSE = 0.4842478381887938
MAE = 0.41441347237666126
Hit rate = 0.4966887417218543
---Running Time: 4.797917604446411 seconds ---
**************************************************
-- Elapsed time -- Epoch -- Cost value -- 
--    43.567145 --  1000 --    0.18229 -- 
-- Making prediction at 1000th epoch
Making prediction.......
0
RMSE = 0.5098532404670443
MAE = 0.43030810644435813
Hit rate = 0.4966887417218543
---Running Time: 4.773400068283081 seconds ---
**************************************************
-- Elapsed time -- Epoch -- Cost value -- 
--    87.299596 --  2000 --    0.20259 -- 
-- Making prediction at 2000th epoch
Making prediction.......
0
RMSE = 0.3797582127584835
MAE = 0.2933602471327154
Hit rate = 0.5006622516556292
---Running Time: 4.988360404968262 seconds ---
*************************************************

In [30]:
df_train_data = helper.legendre_expansion(df_train_d, 8)
df_target = df.loc[df_train_data.index][['Close']]
df_target = helper.inter_polation_norm_1_1(df_target)
## Train test split
x_train, y_train = df_train_data.loc[:'2014-01-01'].as_matrix(), df_target.loc[:'2014-01-01']
x_test  = df_train_data.loc['2014-01-01':].as_matrix()
d = './MLP_decay_leg_L2_1/'
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    optimize(x_train, y_train, x_test, df_target.loc['2014-01-01':] , TRAINING_EPOCHS, BATCH_SIZE, sess, saver, d)
    
    

Training.......
-- Elapsed time -- Epoch -- Cost value -- 
--     0.047524 --     0 --    0.22427 -- 
-- Making prediction at 0th epoch
Making prediction.......
0
RMSE = 0.42504565019956025
MAE = 0.35771446854482153
Hit rate = 0.519205298013245
---Running Time: 4.719496011734009 seconds ---
**************************************************
-- Elapsed time -- Epoch -- Cost value -- 
--    43.771982 --  1000 --    0.05416 -- 
-- Making prediction at 1000th epoch
Making prediction.......
0
RMSE = 0.4703356402006034
MAE = 0.3878506625184866
Hit rate = 0.5165562913907285
---Running Time: 4.8155646324157715 seconds ---
**************************************************
-- Elapsed time -- Epoch -- Cost value -- 
--    88.225945 --  2000 --    0.05576 -- 
-- Making prediction at 2000th epoch
Making prediction.......
0
RMSE = 0.4122324774221451
MAE = 0.3329792623414937
Hit rate = 0.519205298013245
---Running Time: 5.012661695480347 seconds ---
**************************************************